In [ ]:
""" Imaging to understand what happens with opacities and optical depth """

In [ ]:
# Vanilla imports
import numpy as np
import matplotlib.pyplot as plt
import colorcet
import matplotlib.colors as colors

# Chocolate imports
from src.Luminosity.photosphere import get_photosphere
from src.Optical_Depth.opacity_table import opacity

# Constants
c = 2.99792458e10 #[cm/s]
h = 6.62607015e-27 #[gcm^2/s]
Kb = 1.380649e-16 #[gcm^2/s^2K]
alpha = 7.5646 * 10**(-15) # radiation density [erg/cm^3K^4]
Rsol_to_cm = 6.957e10

In [ ]:
# PLOT FOR ELENA
loadpath = 'src/Optical_Depth/'
lnT = np.loadtxt(loadpath + 'T.txt')
lnrho = np.loadtxt(loadpath + 'rho.txt')
lnk_ross = np.loadtxt(loadpath + 'ross.txt')
lnk_planck = np.loadtxt(loadpath + 'planck.txt')
lnrho_extended = np.loadtxt(loadpath + 'rho_expansion.txt')
lnk_ross_extended = np.loadtxt(loadpath + 'ross_expansion.txt')
lnk_planck_extended = np.loadtxt(loadpath + 'planck_expansion.txt')

In [ ]:
# TEST COLORMESH
# def f(x,y):
#     return x+y

# tab = np.zeros((3,2))
# tab[0][0] = 3
# tab[0][1] = 4
# tab[1][0] = 6
# tab[1][1] = 8
# tab[2][0] = 9
# tab[2][1] = 12


# plt.figure(figsize = (4,2))
# img = plt.pcolormesh([1,2,3], [3,4], tab.T,  cmap = 'cet_gouldian')
# cbar = plt.colorbar(img)
# cbar.set_label(r'$k_P$', fontsize = 15)

In [ ]:
plt.figure(figsize = (8,6))
img = plt.pcolormesh(np.exp(lnT), np.exp(lnrho), np.exp(lnk_planck).T, cmap = 'cet_gouldian', norm = colors.LogNorm())
cbar = plt.colorbar(img)
cbar.set_label(r'$k_P$', fontsize = 15)
plt.xlabel('$log_{10}$ T [K]')
plt.ylabel(r'$log_{10}\rho$ [g/$cm^2$]')
img.axes.get_yaxis().set_ticks([])
plt.loglog()
plt.savefig('Figs/k_P.png')
plt.title('Planck opacity')

plt.figure(figsize = (8,6))
img = plt.pcolormesh(np.exp(lnT), np.exp(lnrho), np.exp(lnk_ross).T, cmap = 'cet_gouldian', norm = colors.LogNorm())
cbar = plt.colorbar(img)
plt.title('Rosseland opacity')
cbar.set_label(r'$k_R$', fontsize = 15)
plt.xlabel('$log_{10}$ T [K]')
plt.ylabel(r'$log_{10}\rho$ [g/$cm^2$]')
img.axes.get_yaxis().set_ticks([])
plt.loglog()
plt.savefig('Figs/k_R.png')
plt.title('Rosseland opacity')

In [ ]:
planck_extended = np.exp(lnk_planck_extended)
planck_extended = np.nan_to_num(planck_extended, nan = 0, posinf = 0, neginf= 0)
ross_extended = np.exp(lnk_ross_extended)
ross_extended = np.nan_to_num(ross_extended, nan = 0, posinf = 0, neginf= 0)
rho_extended = np.exp(lnrho_extended)
rho_extended = np.nan_to_num(rho_extended, nan = 0, posinf = 0, neginf= 0)

plt.figure(figsize = (6,3))
img = plt.pcolormesh(np.exp(lnT), rho_extended, planck_extended.T, cmap = 'cet_gouldian', norm = colors.LogNorm())
cbar = plt.colorbar(img)
cbar.set_label(r'$k_P$', fontsize = 15)
plt.xlabel('$log_{10}$ T [K]')
plt.ylabel(r'$log_{10}\rho$ [g/$cm^2$]')
img.axes.get_yaxis().set_ticks([])
plt.loglog()

plt.savefig('Figs/k_P_extended.png')
plt.title('Planck opacity (extended)')

plt.figure(figsize = (8,6))
img = plt.pcolormesh(np.exp(lnT), rho_extended, ross_extended.T, cmap = 'cet_gouldian', norm = colors.LogNorm())
cbar = plt.colorbar(img)
plt.title('Rosseland opacity')
cbar.set_label(r'$k_R$', fontsize = 15)
plt.xlabel('$log_{10}$ T [K]')
plt.ylabel(r'$log_{10}\rho$ [g/$cm^2$]')
img.axes.get_yaxis().set_ticks([])
plt.loglog()
plt.savefig('Figs/k_R_extended.png')
plt.title('Rosseland opacity (extended)')

In [ ]:
# # TEST FOR EXTRAPOLATION
# from scipy.interpolate import CubicSpline

# def extrapolation_table(rho, kind):
#     extra = np.zeros(len(lnT))
#     for i in range(len(lnT)):
#         if kind == 'rosseland':
#             opacity_row = lnk_ross[i]
#         elif kind == 'planck':
#             opacity_row = lnk_planck[i]     
#         elif kind == 'effective':
#             opacity_row = np.add(lnk_planck[i], lnk_scatter[i])
#             opacity_row = np.multiply(3 * opacity_row, lnk_planck[i])
#             opacity_row = np.sqrt(opacity_row)   
#         elif kind == 'red':
#             opacity_row = np.add(lnk_planck[i], lnk_scatter[i])
#         cubicspl = CubicSpline(lnrho, opacity_row, bc_type='natural')
#         extra[i] = cubicspl(rho)
#     print(np.shape(extra))
#     return extra


# if __name__ == '__main__':
#     # Minimum we need is 3.99e-22, Elad's lnrho stops at 1e-10
#     rho_min = np.log(3.99e-22)
#     rho_max = np.log(8e-11)
#     expanding_rho = np.arange(rho_min,rho_max, 0.2)
#     expanding_rho = np.arange(1,5)
#     colum_expanded_rho = len(expanding_rho) + len(lnrho)
#     table_expansion = np.zeros( (len(lnT), colum_expanded_rho ))
#     for i, T in enumerate(lnT):
#         opacity_col = lnk_ross[i] # line to change
#         extra = CubicSpline(lnrho, opacity_col, bc_type='natural')
#         for j, rho in enumerate(expanding_rho):           
#             opi = extra(rho)
#             if opi > 0 :
#                 table_expansion[i,j] = opi
#         for j in range(len(expanding_rho),colum_expanded_rho):
#             table_expansion[i,j] = opacity_col[j-len(expanding_rho)]
    
#     all_rhos = np.concatenate((expanding_rho, lnrho))
#     np.savetxt('testR.txt', all_rhos)
#     np.savetxt('testross.txt', table_expansion)

     